In [1]:
import pandas as pd
import csv

In [2]:
data_path = "Data/sample.tsv"
schema_path = "Data/schema.txt"

In [3]:

sample =  pd.read_csv(data_path, sep="\t",encoding='utf-8',  quoting=csv.QUOTE_NONE)

In [5]:
with open(schema_path) as f:
    columns = f.readlines()

In [6]:
columns[0]

'1    id    bigint(20)        UNSIGNED    No    None\n'

In [7]:
schema = pd.read_csv(schema_path, sep="\s+", header =None)

In [8]:
schema.drop(labels = 0, axis =1, inplace= True)

In [9]:
schema.head()

,1,2,3,4,5
0,id,bigint(20),UNSIGNED,No,None
1,userId,bigint(20),UNSIGNED,No,None
2,createdAt,timestamp,No,0000-00-00,00:00:00
3,text,text,utf8_unicode_ci,No,None
4,longitude,float,Yes,NaN,NaN


In [10]:
sample.columns = schema[1]

In [11]:
sample.head()

1,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,776523000636203010,2741685639,2016-09-15 20:48:05,https://t.co/noYrTnqmg9,\N,\N,4e7c21fd2af027c6,\N,1.0,NaN,46.8131,8.22414,Twitter for iPhone,http://twitter.com/#!/download/iphone,samara,letisieg,755,2037.0,3771.0,Suisse
1,776523045200691200,435239151,2016-09-15 20:48:15,@BesacTof @Leonid_CCCP Tu dois t'engager en si...,\N,\N,12eb9b254faf37a3,776522113859608576,5.0,NaN,47.201,5.94082,Twitter for Android,http://twitter.com/download/android,lebrübrü❤,lebrubru,811,595.0,30191.0,Fontain
2,776523058404290560,503244217,2016-09-15 20:48:18,@Mno0or_Abyat اشوف مظاهرات على قانون العمل الج...,\N,\N,30bcd7f767b4041e,776521597515624448,1.0,NaN,45.8011,6.16552,Twitter for iPhone,http://twitter.com/#!/download/iphone,عبدالله القنيص,bingnais,28433,417.0,12262.0,Shargeyah
3,776523058504925185,452805259,2016-09-15 20:48:18,Greek night #geneve (@ Emilios in Genève) http...,6.14414,46.1966,c3a6437e1b1a726d,\N,3.0,NaN,46.2048,6.14319,foursquare,http://foursquare.com,Alkan Şenli,Alkanoli,204,172.0,3390.0,İstanbul/Burgazada
4,776523071025012736,16416746,2016-09-15 20:48:21,@gregorypons #BusinessMontresVision https://t....,\N,\N,c3a6437e1b1a726d,776520907107995652,18777.0,NaN,46.2048,6.14319,Twitter Web Client,http://twitter.com,Gregory PONS,gregorypons,2398,305.0,14917.0,Geneva + watchmaking planet


In [12]:
sample.shape

(9998, 20)